BPNN模型:
使用随机梯度下降（SGD），sigmoid激活函数，

In [292]:
# 导入numpy库，用于数组和矩阵计算
import numpy as np

# 定义sigmoid激活函数
def sigmoid(x):
    # Sigmoid函数将输入映射到0-1之间
    return 1 / (1 + np.exp(-x))

# 定义sigmoid函数的导数，用于反向传播过程中的梯度计算
def sigmoid_derivative(x):
    # Sigmoid导数计算
    return x * (1 - x)

# 定义BP神经网络类
class BPNN:
    # 初始化函数
    def __init__(self, layers, alpha=0.1,epochs=1000,random_state=None):
        # layers: 各层节点数的列表
        # alpha: 学习率
        self.layers = layers
        self.alpha = alpha
        self.weights = []
        self.epochs=epochs
        self.random_state=random_state
        self.init_weights()
    
        # 初始化权重矩阵
        

    def init_weights(self):
        np.random.seed(self.random_state)
        for i in range(1, len(self.layers)):
            # 随机初始化权重，并设置偏置项
            w = 2 * np.random.random((self.layers[i-1] + 1, self.layers[i] + 1)) - 1
            self.weights.append(w)
        # 调整输出层权重矩阵的维度
        self.weights[-1] = self.weights[-1][:, :-1]
    
    def forward_prop(self,x):
        a = [x]  # 输入层的激活值
        # 遍历每一层，计算激活值
        for l in range(len(self.weights)):
            dot_value = np.dot(a[l], self.weights[l]) 
            activation=sigmoid(dot_value)
            a.append(activation)
        return a
    def backward_prop(self,a,y):
        # 计算误差
        error = y - a[-1]
        # 计算误差梯度
        deltas = [error * sigmoid_derivative(a[-1])]

        # 反向传播
        for l in range(len(a) - 2, 0, -1):
            deltas.append(deltas[-1].dot(self.weights[l].T)*sigmoid_derivative(a[l]))
        deltas.reverse()

        return deltas
    def fit(self, X, y):
        # 添加偏置项
        X = np.c_[X, np.ones((X.shape[0]))]
        # 进行epochs次迭代
        for epoch in range(self.epochs):
            # 随机梯度下降
            i=np.random.randint(X.shape[0])
            a=self.forward_prop(X[i])
            deltas=self.backward_prop(a,y[i])
            # 更新权重
            for j in range(len(self.weights)):
                layer = np.atleast_2d(a[j])
                delta = np.atleast_2d(deltas[j])
                self.weights[j] += self.alpha * layer.T.dot(delta)

    # 预测函数
    def predict(self, X_test):
        predictions = []
        for x in X_test:
        # 添加偏置项
            a = np.concatenate((np.array(x), np.ones(1)))
            # 正向传播
            for l in range(0, len(self.weights)):
                a = sigmoid(np.dot(a, self.weights[l]))
            pred=np.argmax(a)
            predictions.append(pred)
        
        return predictions


使用sklearn中的鸢尾花数据集，进行训练

In [293]:
'''鸢尾花数据集
    150个样本,按8:2划分为训练集和测试集
    3个类别('Iris-setosa', 'Iris-versicolor', 'Iris-virginica'),
    4个属性('sepal length','sepal width','petal length','petal width'),
    构建决策树分类模型并测试准确率
    呈现训练过程,绘制分类结果,评估模型准确性
'''
# 使用鸢尾花数据集进行验证
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 加载数据
iris = load_iris()
X = iris.data
y = iris.target

# 将标签转换为独热编码
y = np.eye(3)[y]

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

# 创建BPNN实例
bpnn = BPNN([4, 6, 3],alpha=0.01,epochs=10000,random_state=42)

# 训练网络
bpnn.fit(X_train, y_train)

# 预测测试集
predictions=bpnn.predict(X_test)

# 计算准确率
accuracy = accuracy_score(np.argmax(y_test, axis=1), predictions)
print(f'Accuracy: {accuracy}')


Accuracy: 1.0


使用UCI数据集中的葡萄酒质量数据集,进行训练

In [294]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
wine_quality = fetch_ucirepo(id=186) 
  
# data (as pandas dataframes) 
X = wine_quality.data.features 
y = wine_quality.data.targets 


X=np.array(X)
y=np.array(y)
y=y.flatten()

y=np.eye(10)[y]
# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

# 创建BPNN实例
bpnn = BPNN([11,25,10],alpha=0.01,epochs=10000,random_state=42)

# 训练网络
bpnn.fit(X_train, y_train)

# 预测测试集
predictions=bpnn.predict(X_test)
# 计算准确率

accuracy = accuracy_score(np.argmax(y_test, axis=1), predictions)
print(f'Accuracy: {accuracy}')

Accuracy: 0.46
